In [41]:
def openalex_name(author):
    import requests, time
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        if nb_openalex>0:
            d=[]
            for n in range(nb_openalex):
                author_openalex = requests.get(url).json().get("results")[n]
                print(author_openalex)
                result = author | {'display_name':author_openalex.get('display_name'),
                                'openalex_id':author_openalex.get('id'), 
                                'affiliations':author_openalex.get('affiliations'), 
                                'topics':author_openalex.get('topics'), 
                                'x_concepts':author_openalex.get('x_concepts'), 
                                'ids':author_openalex.get('ids'), 
                                'display_name_alternatives':author_openalex.get('display_name_alternatives')}
                d.append(result)
        return d

    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")                    
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
    

In [42]:
def openalex_orcid(author):
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=zmenesr@gmail.com"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives')}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")                    
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
    

In [40]:
import pandas as pd
df=pd.DataFrame()

In [ ]:
import pandas as pd, time, re, numpy as np, requests, pprint as pp
author = {
"name": 'jeremy peglion',
"orcid": ''
}
# try:
if author.get("orcid"):
    result = openalex_orcid(author)
    if result:
        df=pd.concat([df, pd.json_normalize(result)])
    else:
        result = openalex_name(author)
        if result:
            df=pd.concat([df, pd.json_normalize(result)])
if author.get("orcid")=='':
    result = openalex_name(author)
    if result:
        df=pd.concat([df, pd.json_normalize(result)])

# except requests.exceptions.HTTPError as http_err:
#     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
# except requests.exceptions.RequestException as err:
#     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")                    
# except Exception as e:
#     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")

In [ ]:
df

In [ ]:
def prep(stage, df, countries, lien):

    test = df.merge(countries[['countryCode', 'country_code_mapping','country_code']], how='left', on='countryCode')
    test = test.assign(stage=stage).drop(columns=['countryCode','orderNumber', 'departmentUniqueId','framework', 'lastUpdateDate' ]).drop_duplicates()
#     test['nb'] = test.groupby(['project_id', 'generalPic', 'pic'])['department'].transform('count')

    if stage=='evaluated':
        tmp=(lien.loc[lien.inProposal==True, ['project_id', 'generalPic', 'proposal_orderNumber','proposal_participant_pic', 'calculated_pic', 'nuts_applicants', 'n_app']]
            .rename(columns={'nuts_applicants':'entities_nuts', 'proposal_participant_pic':'pic', 'proposal_orderNumber':'orderNumber', 'n_app':'ent_nb'}))
        tmp=tmp.merge(test, how='inner', on=['project_id',  'generalPic',  'pic'])
    elif  stage=='successful':
        tmp=(lien.loc[lien.inProject==True, ['project_id', 'generalPic', 'orderNumber', 'participant_pic', 'calculated_pic', 'nuts_participant', 'n_part']]
        .rename(columns={'nuts_participant':'entities_nuts', 'participant_pic':'pic', 'n_part':'ent_nb'}))
        tmp=tmp.merge(test, how='inner', on=['project_id',  'generalPic',  'pic'])
    
    tmp.entities_nuts=tmp.apply(lambda x: ','.join(x.strip() for x in x.entities_nuts if x.strip()), axis=1)
    return tmp.sort_values('project_id').drop_duplicates()

#######
app=prep('evaluated', pp_app, countries, lien)
part=prep('successful', pp_part, countries, lien)
print(f"app {len(app)}, part {len(part)}")

lp = part[['project_id', 'generalPic', 'pic', 'country_code_mapping']].drop_duplicates()
app = app.merge(lp, how='left', indicator=True).query('_merge=="left_only"').drop(columns='_merge')

#######
struct = pd.concat([app, part], ignore_index=True)
struct['nb_stage'] = struct.groupby(['project_id', 'generalPic', 'country_code', 'orderNumber','calculated_pic','stage'])['department'].transform('count')
struct = (struct
            .rename(columns={'country_code_mapping':'country_code_mapping_dept', 'country_code':'country_code_dept', 'nutsCode':'department_nuts'}))
print(f"size structure {len(struct)}")

In [ ]:
if len(participation[['stage','project_id','generalPic','orderNumber', 'country_code','country_code_mapping']].drop_duplicates())!=len(participation[['stage','project_id','generalPic','orderNumber', 'country_code','country_code_mapping','role','participates_as']].drop_duplicates()):
    print("Attention doublon d'une participation avec ajout de role+participates_as")

# merge struct (department) with participation
part = participation[['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage']].drop_duplicates()
print(f"size part {len(part)}")
part1 = (part
        .merge(struct, 
            how='inner', 
            left_on=['stage','project_id', 'generalPic', 'orderNumber','country_code_mapping'],  
            right_on=['stage','project_id', 'generalPic', 'orderNumber', 'country_code_mapping_dept'])
        .drop_duplicates())
print(f"size part1 merge_on['stage','project_id', 'generalPic', 'orderNumber','country']:{len(part1)}")

part2 = (part.merge(part1[['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage']], 
                    how='outer', on=['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage'], indicator=True)
        .query('_merge == "left_only"')
        .drop('_merge', axis=1)
        .drop_duplicates()
        .merge(struct, 
            how='inner', 
            left_on=['stage','project_id', 'generalPic', 'orderNumber'],  
            right_on=['stage','project_id', 'generalPic', 'orderNumber'])
        .drop_duplicates())
print(f"size part2 merge_on['stage','project_id', 'generalPic', 'orderNumber']:{len(part2)}")

part2 = pd.concat([part1, part2], ignore_index=True, axis=0)

part3 = (part.merge(part2[['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage']], 
                    how='outer', on=['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage'], indicator=True)
        .query('_merge == "left_only"')
        .drop('_merge', axis=1)
        .merge(struct.drop(columns='generalPic'), 
            how='inner', 
            left_on=['stage','project_id', 'orderNumber'],  
            right_on=['stage','project_id', 'orderNumber'])
        .drop_duplicates())
print(f"size part3 merge_on['stage','project_id','orderNumber']:{len(part3)}")

part3 = pd.concat([part2, part3], ignore_index=True, axis=0)
print(f"size part3:{len(part3)}")

part4 = (part.merge(part3[['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage']], 
                    how='outer', on=['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage'], indicator=True)
        .query('_merge == "left_only"')
        .drop('_merge', axis=1)
        .merge(part3.drop(columns='stage'), 
               how='inner', on=['project_id','generalPic','orderNumber', 'country_code','country_code_mapping']))

print(f"size part4 merge_on with part3 ['project_id','generalPic','orderNumber', 'country_code','country_code_mapping']:{len(part4)}")

part4 = pd.concat([part4, part3], ignore_index=True, axis=0)
print(f"size part4:{len(part4)}")

part5 = (part.merge(part4[['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage']], 
                    how='outer', on=['project_id','generalPic','orderNumber', 'country_code','country_code_mapping','stage'], indicator=True)
        .query('_merge == "left_only"')
        .drop('_merge', axis=1))

print(f"size part5 reste participation:{len(part5)}")

part5 = pd.concat([part4, part5], ignore_index=True, axis=0)
print(f"size part5:{len(part5)}")

part5['nb'] = part5.groupby(['stage', 'project_id', 'generalPic', 'orderNumber'])['stage'].transform('count')
part5['nb2'] = part5.groupby(['stage', 'project_id', 'generalPic', 'orderNumber','country_code_mapping'])['stage'].transform('count')
part5[['country_code','country_code_mapping']] = part5[['country_code','country_code_mapping']].fillna(part5.groupby(['stage', 'project_id', 'generalPic', 'orderNumber'])[['country_code','country_code_mapping']].ffill())
print(f"size part5 {len(part5)}")

# #remove participation duplicates on ['stage', 'project_id', 'generalPic', 'orderNumber'] and department empty
part5 = part5.loc[~((part5.nb>1)&(part5.department.isnull()))]
print(f"size part5 {len(part5)}")


In [ ]:
structure = (part5
            .merge(entities_info[['generalPic', 'legalName', 'businessName',
            'category_woven', 'city', 'country_code_mapping', 'country_code',  'country_name_fr', 
            'id_secondaire', 'entities_id', 'entities_name',  'entities_acronym', 'operateur_num', 'postalCode', 
            'street', 'webPage']], 
            how='left', on=['generalPic', 'country_code_mapping', 'country_code'])
            .merge(proj[['project_id', 'call_year']].drop_duplicates(), how='left', on=['project_id'])
            .drop(columns=['nb_stage', 'nb', 'nb2'])
            .drop_duplicates()
            )
print(len(structure))

In [ ]:
structure = structure.loc[~structure.entities_name.isnull()].drop_duplicates()
print(len(structure))

cols = ['department', 'entities_acronym', 'entities_name', 'legalName', 'businessName']
for i in cols:
    structure[f"{i}_dup"] = structure.loc[:,i]

if any(structure.call_year.isnull()):
    print(f"vérification de l'année (corriger les nuls si existants):\n{structure.call_year.value_counts(dropna=False)}")

##########
cols = ['department_dup', 'legalName_dup', 'businessName_dup', 'entities_acronym_dup','entities_name_dup','street','city']
structure = prep_str_col(structure, cols)

cedex="cedax|cedrex|cdexe|cdex|credex|cedex|cedx|cede|ceddex|cdx|cex|cexex|edex"
structure.loc[structure.postalCode.isnull(), 'postalCode'] = structure.city.str.extract(r"(\d+)")
structure['city'] = structure.city.str.replace(r"\d+", ' ', regex=True).str.strip()
structure.loc[structure.country_code=='FRA', 'city'] = structure.city.str.replace(cedex, ' ', regex=True).str.strip()
structure.loc[structure.country_code=='FRA', 'city'] = structure.city.str.replace(r"^france$", '', regex=True).str.strip()

##########
# creation entities_full = entities_name + entities_acronym et department_tag
tmp = structure.loc[(structure.legalName_dup!=structure.businessName_dup)&(~structure.businessName_dup.isnull()), ['generalPic',  'country_code', 'legalName_dup', 'businessName_dup']]
tmp['entities_full'] = [x1 if x2 in x1 else x1+' '+x2 for x1, x2 in zip(tmp['legalName_dup'], tmp['businessName_dup'])]

if len(structure.drop_duplicates())!=len(structure.merge(tmp[['generalPic', 'country_code', 'legalName_dup', 'businessName_dup', 'entities_full']].drop_duplicates(), how='left', on=['generalPic','businessName_dup', 'legalName_dup','country_code']).drop_duplicates()):
    print("Attention risque de doublon si merge de tmp et structure")
else:
    structure = structure.merge(tmp[['generalPic', 'country_code','legalName_dup', 'businessName_dup', 'entities_full']].drop_duplicates(), how='left', on=['generalPic','legalName_dup', 'businessName_dup', 'country_code']).drop_duplicates()
    structure.loc[structure.entities_full.isnull(), 'entities_full'] = structure.entities_name_dup.str.lower()

#############
societe = pd.read_table('data_files/societe.txt', header=None)
structure.loc[structure.entities_full.apply(lambda x: True if re.search(r"(?=\b("+'|'.join(list(set(societe[0])))+r")\b)", x) else False), 'org1'] = 'societe'
societe = societe.loc[societe[0]!='group']
structure.loc[(~structure.department_dup.isnull())&(structure.department_dup.apply(lambda x: True if re.search(r"(?=\b("+'|'.join(list(set(societe[0])))+r")\b)", str(x)) else False)), 'org1'] = 'societe'
structure.loc[structure.category_woven=='Entreprise', 'org1'] = 'societe'

las = r"(\bas(s?)ocia[ctz][aionj]+)|\b(ev|udruga|sdruzhenie|asbl|aisbl|vzw|biedriba|kyokai|mittetulundusuhing|ry|somateio|egyesulet(e?)|stowarzyszenie|udruzenje|zdruzenie|sdruzeni(e?))\b|([a-z]*)(verband|vereniging|asotsiatsiya|zdruzenje)\b|([a-z]*)(verein|forening|yhdistys)([a-z]*)"
structure.loc[structure.entities_full.apply(lambda x: True if re.search(las , x) else False), 'org2'] = 'association'
structure.loc[structure.category_woven=='Institutions sans but lucratif (ISBL)', 'org2'] = 'association'

structure['typ_from_lib'] = structure[['org1','org2']].stack().groupby(level=0).agg(' '.join)
structure.drop(columns=['org1','org2'], inplace=True)

# mots vide à suppr
stop_word(structure, 'country_code', ['entities_full', 'department_dup'])

structure['entities_full'] = structure['entities_full_2'].apply(lambda x: ' '.join([s for s in x if s.strip()]))
structure.loc[(~structure.department_dup.isnull()), 'department_dup'] = structure.loc[(~structure.department_dup.isnull()), 'department_dup_2'].apply(lambda x: ' '.join([s for s in x if s.strip()]))

structure.drop(columns=['department_dup_2', 'entities_full_2'], inplace=True)
structure.mask(structure=='', inplace=True)



In [ ]:
##################################
#########################
#################
### FRANCE


structure_fr = structure.loc[structure.country_code=='FRA']
print(len(structure_fr))


#############


def qualif_organisation(x):
    lpattern = ["cnrs", "inria", "inrae", "ifremer", "inserm", "cea", "ens", "fnsp", "cirad", "ird", "chu", "universite", 
            "pasteur", "curie", "irsn", "onera", "agrocampus", "ed","ecole"]
    pattern_ifremer = "(ifremer)|(in.* fran.* re.* ex.* mer)"
    pattern_cnrs =   "(ce.* na.* (de )?(la )?re.* sc.[a-z]*)|(fr.* na.* sc.* re.* ce.[a-z]*)|(cnrs)"
    pattern_inria =  "(in.* na.* (de )?re.* (en )?in.* (et )?(en )?au.[a-z]*)|(inria)"
    pattern_inrae =   "(in.* na.* (de )?re.* ag.[a-z]*)|(inra)|(inrae)|(irstea)"
    pattern_inserm = "(in.* na.* (de )?(la )?sa.* (et )?(de )?(la )?re.* me.[a-z]*)|(inserm)"
    pattern_cea =    "(co.* (a )?l?\\'?en.* at.[a-z]*)|(\\bcea\\b)"
    pattern_ens =    "(ec.* no.* sup[a-z]*)|(\\bens\\b)"
    pattern_fnsp =   "(fo.* na.* (des )?sc.* po.[a-z]*)|(fnsp)|(sciences po)"
    pattern_cirad =  "(ce.* (de )?co.* in.* (en )?re.* ag.* (pour )?(le )?dev.[a-z]*)|(cirad)"
    pattern_ird =    "(in.* (de )?re.[a-z]* (pour )?(le )?dev.[a-z]*)|\\b(ird)\\b|(i r d)"
    pattern_chu = "((ce.*|ctre|group.*) hos.* (univ.[a-z]*)?)|(univ.* hosp.[a-z]*)|\\b(chu|chr|chru)\\b|(hospice)"
    pattern_universite =   "(univ(ersite|ersity|ersitaire))"
    pattern_pasteur =   "(ins([a-z]*|\\.*) pasteur( de)?( lille)?)|(pasteur inst([a-z]*))"
    pattern_curie =    "(inst([a-z]*|\\.*) curie)|(curie inst([a-z]*))"
    pattern_irsn =   "(in.* (de )?radio.[a-z]* (et )?(de )?sur.[a-z]* nuc.[a-z]*)|(irsn)"
    pattern_onera =  "(onera)|(off.* na.* (d )?etu.* (et )?(de )?rech.* aero.*)"
    pattern_agrocampus = "(agrocampus)"
    pattern_ed = "(doct.* sch.*)|(ec.* doct.*)|\\b(ed)\\b"
    pattern_ecole = "(ecole)"
    org = []
    for j in lpattern:
        pattern = globals()[f"pattern_{j}"]
        y = re.search(pattern, x)
        if y:
            org.append(j)
    return org

structure_fr['org1'] = structure_fr.apply(lambda x: qualif_organisation(x['department_dup']) if isinstance(x['department_dup'], str) else [], axis=1)
structure_fr['org2'] = structure_fr.apply(lambda x: qualif_organisation(x['entities_full']) if isinstance(x['entities_full'], str) else [], axis=1)
structure_fr['org3'] = structure_fr.apply(lambda x: qualif_organisation(x['entities_name_dup']) if isinstance(x['entities_name_dup'], str) else [], axis=1)


structure_fr['org_from_lib'] = structure_fr.apply(lambda x: sorted(set(x['org1'] + x['org2'] + x['org3'])), axis=1)
# structure_fr['org_from_lib'] = structure_fr['org_from_lib'].apply(lambda x: ' '.join(x))

structure_fr.drop(columns=['org1', 'org2', 'org3'], inplace=True)
structure_fr.mask(structure_fr=='', inplace=True)

In [3]:


    


    structure_fr=structure_fr.assign(dep_tag=structure_fr.department_dup, lab_tag=structure_fr.entities_full)
    cols = ['dep_tag', 'lab_tag']
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace('international research lab', "irl", regex=False))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace('joint research unit', "jru", regex=False))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace('joint research unit', "jru", regex=False))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace('equipe accueil', "ea", regex=False))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace(r"\bumr(\s?s\s?)(u(\s?)|inserm(\s?))?(?=(\d+)?)|\bu\\s?inserm(\s?)|\bunit(e?)(?=(\s?u?\s?\d+))|\binserm\s?(umr\s?(s?)|jru)\s?(u?)|\binserm(u?)\s?(?=\d+)|\binserm\s?un\s?umr\s?u?", "u", regex=True))
    for s in ['umr','upr','uar','irl','emr','umi','usr','fre','gdr','fr']:
        structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace(r"(?<=\b"+s+r")\s?[a-z]+\s?(?=\d+)", " ", regex=True))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace(r"\bu\s?cnrs|\bum\s+r|\bcnrs\s?(?=\d+)|\bjru\s?(cnrs|umr)", "umr", regex=True))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace(r"\bjru\s?(umi)", "umi", regex=True))
    structure_fr[cols] = structure_fr[cols].apply(lambda x: x.str.replace(r"(\bce[a-z]* inv[a-z]* cl[a-z]*)|(\bcl[a-z]* inv[a-z]* ce[a-z]*)|(\bce[]* cl[a-z]* inv[a-z]*)", "cic", regex=True))
    structure_fr.loc[structure_fr.org_from_lib.map(lambda x: "inserm" in x), cols] = structure_fr.loc[structure_fr.org_from_lib.map(lambda x: "inserm" in x), cols].apply(lambda x: x.str.replace(r"\bjru\b", 'u', regex=True))
    structure_fr.loc[structure_fr.org_from_lib.map(lambda x: "cnrs" in x), cols] = structure_fr.loc[structure_fr.org_from_lib.map(lambda x: "cnrs" in x), cols].apply(lambda x: x.str.replace(r"\bjru\b", 'umr', regex=True))

    llab = ["umr", "ua", "umrs", "umr s","ea", "u", "gdr", "fre", "fr", "frc", "fed", "je", "us", "ums",
            "upr","upesa","ifr","umr a","umemi","epi","eac", "ertint", "ur", "ups", "umr m", "umr t",
            "uar","ert","usr","ura","umr d","rtra","ue","ers","cic","ep","umi", "unit", 'emr', 'irl', 'jru']

    def labo_sigle(x):
        sig = []
        for i in llab:
            pattern = r"\b("+i+r")(?=\b|\d+)\s?[a-z]*\s?(\d+)"
            y = re.search(pattern, x)
            if y:
                sig.append(''.join(y.groups()))
        return sig

    structure_fr['org1'] = structure_fr.apply(lambda x: labo_sigle(x['dep_tag']) if isinstance(x['dep_tag'], str) else [], axis=1)
    structure_fr['org2'] = structure_fr.apply(lambda x: labo_sigle(x['lab_tag']) if isinstance(x['lab_tag'], str) else [], axis=1)
    structure_fr['lab_from_lib'] = structure_fr.apply(lambda x: list(set(x['org1'] + x['org2'])), axis=1)
    # structure_fr['lab_from_lib'] = structure_fr['lab_from_lib'].apply(lambda x: ';'.join(x))
    structure_fr.drop(columns=['org1', 'org2', 'dep_tag', 'lab_tag'], inplace=True)
    structure_fr.mask(structure_fr=='', inplace=True)

    print(f"size structure_fr: {len(structure_fr)}")

    #######################################################
    #RETOUR ORGANISMES

    organisme_back = pd.read_pickle(f"{PATH_ORG}organisme_back.pkl").drop_duplicates()
    organisme_back = organisme_back.drop(columns=['lib_back', 'location_back']).drop_duplicates()
    print(len(organisme_back))

    stage_proj = structure_fr[['stage', 'project_id']].drop_duplicates()
    organisme1 = organisme_back.merge(stage_proj, how='inner', on='project_id').drop(columns=['proposal_orderNumber']).query("stage=='successful'").drop_duplicates()
    print(len(organisme1))
    organisme_back.loc[organisme_back.proposal_orderNumber.isnull(), 'proposal_orderNumber'] = organisme_back.orderNumber
    organisme2 = organisme_back.merge(stage_proj, how='inner', on='project_id').drop(columns=['orderNumber']).rename(columns={'proposal_orderNumber':'orderNumber'}).query("stage=='evaluated'").drop_duplicates()
    print(len(organisme2))
    oback = pd.concat([organisme1, organisme2], ignore_index=True)
    oback = (oback.groupby(['stage','project_id', 'generalPic', 'pic', 'orderNumber'], dropna=False)
            .agg(lambda x: ';'.join(map(str, filter(None, x.dropna().unique())))).reset_index())

    oback[['labo_back', 'org_back']] = oback[['labo_back', 'org_back']].apply(lambda x: x.str.lower())
    # oback['labo_back'] = oback.labo_back.str.split(';').tolist()
        
    oback.mask(oback=='', inplace=True)

    ###########################################################################################################

    # MERGE ORGANISMES ET STRUCTURE

    tmp = structure_fr.merge(oback, how='outer', on=['stage','project_id','generalPic', 'orderNumber'], indicator=True)
    keep = tmp.loc[tmp._merge!='right_only'] #suppr les lignes oback en +

    # traitement cea sans orderNumber
    tmp = tmp.loc[(tmp._merge=='right_only')&(tmp.org_back=='CEA')]
    cea = keep[(keep.generalPic=='999992401')].drop(columns='_merge').sort_values(['project_id','generalPic'])
    print(len(cea))
    cea = cea.drop(columns=list(oback.columns.difference(structure_fr.columns)))
    cea = cea.merge(oback[oback.generalPic=='999992401'].dropna(axis=1, how='all'), how='left', on=['stage','project_id','generalPic'], indicator=True)
    keep = keep[keep.generalPic!='999992401']
    keep = pd.concat([keep, cea], ignore_index=True).drop(columns=['pic', 'role', 'participates_as']).sort_values(['project_id', 'stage', 'generalPic'])

    for i in ['rnsr_back','labo_back','org_back', 'city_back']:
        keep[i] = keep[i].apply(lambda x: x.split(';') if isinstance(x, str) else [])  

    print(len(keep))

    keep['org_merged'] = keep.apply(lambda x: list(set(x['org_back'] + x['org_from_lib'])), axis=1)
    keep.mask(keep=='', inplace=True)

    keep['lab_merged'] = keep.apply(lambda x: list(set(x['labo_back'] + x['lab_from_lib'])), axis=1)

    pattern=r'^[0-9]{9}[A-Z]{1}($|;)'
    keep.loc[keep.id_secondaire.str.contains('0', na=True), 'id_secondaire'] = np.nan
    keep['id_secondaire'] = keep['id_secondaire'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
    keep['rnsr_merged'] = keep.id_secondaire.apply(lambda x: [i for i in x if re.search(pattern, i)])
    keep['rnsr_merged'] = keep.apply(lambda x: list(set(x['rnsr_merged'] + x['rnsr_back'])), axis=1)
    keep.mask(keep=='', inplace=True)

    ################
    labo = keep.loc[((keep.org_merged.str.len() > 0)|(~keep.operateur_num.isnull())), 
            ['call_year','stage','project_id', 'generalPic', 'entities_full', 'department_dup',
        'typ_from_lib', 'org_merged', 'rnsr_merged', 'lab_merged',
        'cp_back', 'city_back', 'operateur_num','category_woven']]
    print(len(labo))

    lab_a_ident = (labo.loc[(keep.rnsr_merged.str.len() == 0), 
                            ['project_id', 'generalPic', 'call_year','department_dup','entities_full','lab_merged', 'city_back']]
                )

    # lab_a_ident['org_merged'] = lab_a_ident['org_merged'].astype(str)          
    lab_a_ident = (lab_a_ident
                    .set_index(['call_year','department_dup','entities_full'])
                    .explode('lab_merged').explode('city_back')
                    .reset_index()
                    .drop_duplicates()
                )
    print(len(lab_a_ident))

    #################
    # 1er step matching by id
    ident_by_id = lab_a_ident.loc[~lab_a_ident.lab_merged.isnull(), ['call_year', 'entities_full', 'lab_merged', 'city_back']].drop_duplicates()
    # ident_by_id

    #######
    org = ident_by_id.rename(columns={"city_back": "city", "lab_merged": "labo", 'entities_full':'supervisor'})
    for f in ['supervisor', 'labo', 'city']:
        org[f] = org[f].fillna('')
    org.head()

    df=org.assign(match = None)
    lab_id=pd.DataFrame()

    ######

    typ="rnsr"
    now = time.strftime("%H:%M:%S")

    for i, row in df.iterrows():
        query="{} {} {}".format(row['city'], row['labo'], row['supervisor'])
        strategies = [[['rnsr_code_number', 'rnsr_supervisor_name', 'rnsr_city']],
                    [['rnsr_code_number', 'rnsr_supervisor_name']],
                    [['rnsr_code_number', 'rnsr_city']]]
        matcher(df, i, typ, query, strategies, year=row['call_year'])

    ###

    x=df.loc[df.match.isnull()]
    lab_id=pd.concat([df.loc[~df.match.isnull()], lab_id], ignore_index=True)
    print(len(lab_id))

    lab_id['match2'] = lab_id['match'].astype(str)
    lab_id['match2'] = lab_id.groupby('labo', as_index=False).pipe(lambda x: x['match2'].transform('nunique'))
    if len(lab_id.loc[lab_id.match2>1, ['labo','match']].sort_values('labo'))>1:
        print('un même identifiant de labo a ++ de rnsr: a vérifier')
    else: print('ok')

    #save first step into matcher
    lab_id = lab_id.drop(columns=['q', 'match2'])
    lab_id.mask(lab_id=='', inplace=True)
    work_csv(lab_id, 'ident_lab1')

    #####
    # 2d step matching by name
    lab_ident1 = lab_a_ident.merge(lab_id, how='left', left_on=['entities_full', 'call_year', 'lab_merged', 'city_back'], right_on=['supervisor', 'call_year', 'labo', 'city'], indicator=True)
    lab_a_ident = lab_ident1.loc[lab_ident1._merge=='left_only'].drop(columns=['_merge', 'match', 'labo', 'city', 'supervisor'])
    ident_by_lib = lab_a_ident[['call_year', 'department_dup', 'lab_merged', 'entities_full', 'city_back']]
    ident_by_lib['labo'] = ident_by_lib[[ 'department_dup', 'lab_merged']].stack().groupby(level=0).apply(lambda x: ' '.join(x))
    org = ident_by_lib.rename(columns={"city_back": "city", 'entities_full':'supervisor'}).drop_duplicates()
    print(len(org))
    for f in ['supervisor', 'labo', 'city']:
        org[f] = org[f].fillna('')

    df=org.assign(match = None)

    #######################
    typ="rnsr"
    now = time.strftime("%H:%M:%S")

    for i, row in df.iterrows():
        query="{} {} {}".format(row['city'], row['labo'], row['supervisor'])

        strategies = [
            [['rnsr_acronym', 'rnsr_name', 'rnsr_supervisor_name']],
    #                   [['rnsr_acronym', 'rnsr_name' 'rnsr_supervisor_name']],
    #                   [['rnsr_name', 'rnsr_supervisor_name', 'rnsr_city']],
                    [[ 'rnsr_name', 'rnsr_supervisor_name']],
                    [['rnsr_acronym', 'rnsr_supervisor_name']]
        ]
        matcher(df, i, typ, query, strategies, year=row['call_year'])

    x=df.loc[df.match.isnull()]
    lab_id=pd.concat([df.loc[~df.match.isnull()], lab_id], ignore_index=True)

    lab_id = lab_id.drop(columns=['q'])
    lab_id.mask(lab_id=='', inplace=True)
    lab_id.to_pickle(f"{PATH_WORK}match_lab2.pkl", compression='gzip')

    # lab_id=pd.read_csv(f"{PATH_WORK}ident_lab2.csv", sep=';')
    # lab_id['call_year']=lab_id['call_year'].astype(str)
    # lab_id['match'] = lab_id.match.apply(lambda x: ast.literal_eval(x))
    ####################################################

    lab_ident2 = lab_a_ident.merge(lab_id, how='left', left_on=['entities_full', 'call_year', 'lab_merged', 'city_back', 'department_dup'], right_on=['supervisor', 'call_year',  'lab_merged', 'city', 'department_dup'], indicator=True)
    lab_a_ident = lab_ident2.loc[lab_ident2._merge=='left_only'].drop(columns=['_merge', 'match', 'labo', 'city', 'supervisor'])
    lab_a_ident = lab_a_ident.loc[~lab_a_ident.department_dup.isnull()]
    print(len(lab_a_ident))

    df=structure.loc[structure.country_code!='FRA', ['entities_full', 'country_code']].drop_duplicates()
    df=df.fillna('')

In [14]:
entities_tmp=entities_all.loc[((entities_all.country_code=='FRA')&(entities_all.rnsr_merged.str.len()==0))|((entities_all.country_code!='FRA')&(entities_all.entities_id.str.contains('pic'))), ['project_id','generalPic','country_code', 'entities_id', 'entities_name']]
perso = perso[['call_year', 'thema_name_en', 'destination_name_en' ,'project_id', 'country_code', 'generalPic', 'title', 'last_name', 'first_name', 'tel_clean', 'domaine_email', 'contact', 'orcid_id']]

pp = perso[['project_id','generalPic' ,'contact', 'orcid_id']].drop_duplicates().merge(entities_tmp, how='inner', on=['project_id','generalPic'])
# pp.mask(pp=='', inplace=True)
pp = pp.fillna('')
# pp = pp.loc[pp.orcid_id=='', ['contact', 'orcid_id']].drop_duplicates()

In [ ]:
def openalex_name(author):
    url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
    nb_openalex=requests.get(url).json().get("meta").get('count')
    result={}
    if nb_openalex>0:
        for n in range(nb_openalex): 
            author_openalex = requests.get(url).json().get("results")[n]
            result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
            author.update(result)
            df=pd.concat([df, pd.json_normalize(author)])
    return df

In [71]:
import requests, pandas as pd, time
df=pd.DataFrame()
author = {
"name": 'elyaakoubi mustapha',
# "name": 'jeremy peglion',
"orcid": ''
}
try:
    if author.get("orcid"):
    # Get author by Orcid
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=bso@recherche.gouv.fr"
        author_openalex = requests.get(url).json().get("results")
        author.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
        df=pd.concat([df, pd.json_normalize(author)])
    else:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        result={}
        if nb_openalex>0:
            for n in range(nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
                author.update(result)
                df=pd.concat([df, pd.json_normalize(author)])
except requests.exceptions.HTTPError as http_err:
    print(f"\n{time.strftime("%H:%M:%S")}-> HTTP error occurred: {http_err}")
except requests.exceptions.RequestException as err:
    print(f"\n{time.strftime("%H:%M:%S")}-> Error occurred: {err}")                    
except Exception as e:
    print(f"\n{time.strftime("%H:%M:%S")}-> An unexpected error occurred: {e}")

In [ ]:
df

In [ ]:
if author_openalex:
        # author.update({'openalex_id':author_openalex.get('id'),'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'orcid_tmp':author_openalex.get('orcid'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
        result.append(author_openalex)
# pd.json_normalize(result)
result

In [ ]:
# perso.orcid_id.value_counts()
x=perso[['project_id','generalPic' ,'contact', 'orcid_id']].drop_duplicates().merge(entities_tmp, how='inner', on=['project_id','generalPic'])
# x=perso[['project_id','generalPic' ,'contact', 'orcid_id']].drop_duplicates()
x.orcid_id.value_counts()
len(x)

In [ ]:


print("\n### FP7")
def call_api():
    call=pd.read_json(open(f"data_files/FP7_calls.json", 'r+', encoding='utf-8'))
    call = pd.DataFrame(call)
    call['call_budget'] = call['call_budget'].str.replace(',', '').astype('float')
    return call
call=call_api()

def ref_select(FP):
    ref_source = ref_source_load('ref')
    # traitement ref select le FP, id non null ou/et ZONAGE non null
    ref = ref_source_2d_select(ref_source, FP)
    return ref
ref, genPic_to_new=ref_select('FP7')

def FP7_load():
    FP7_PATH=f'{PATH}FP7/2022/'
    _FP7 = pd.read_pickle(f"{FP7_PATH}FP7_data.pkl")
    _FP7.rename(columns={'name_source':'legalName', 'acronym_source':'businessName'}, inplace=True)
    print(f"size _FP7 load: {len(_FP7)}\n{_FP7.columns}")
    return _FP7
_FP7=FP7_load()

In [ ]:


country = pd.read_csv(f"{PATH_SOURCE}H2020/country_current.csv", sep=';', encoding='utf-8')
def FP7_cleaning(_FP7, country):
    _FP7 = _FP7.loc[~_FP7.status_code.isin(['INELIGIBLE','WITHDRAWN'])]
    _FP7.loc[_FP7.status_code=='Project Closed', 'status_code'] = 'CLOSED'
    _FP7.loc[_FP7.status_code=='Project Terminated', 'status_code'] = 'TERMINATED'

    _FP7.loc[_FP7.participant_type_code=='N/A', 'participant_type_code'] = 'NA'
    _FP7['role'] = _FP7['role'].str.lower()
    _FP7.loc[_FP7.role=='participant', 'role'] = 'partner'
    _FP7['coordination_number']=np.where(_FP7['role']=='coordinator', 1, 0)
    _FP7.loc[(_FP7.generalPic=='998133396')&(_FP7.countryCode=='ZZ'), 'country_code_mapping'] = 'USA' # bristol meyer
    print(f"- size _FP7 after clean status: {len(_FP7)}, size with id: {len(_FP7.loc[~_FP7.id.isnull()])}")
    
    zz = _FP7.loc[(_FP7.country_code_mapping=='ZZZ')]
    print(f"- size _FP7 sans country_code: {len(zz)}")
    zz = ref.loc[ref.generalPic.isin(zz.generalPic.unique())]
    _FP7 = _FP7.merge(zz, how='left', on='generalPic', suffixes=['','_ref'])
    for i in ['id', 'country_code_mapping', 'ZONAGE']:
        _FP7.loc[~_FP7[f"{i}_ref"].isnull(), i] = _FP7[f"{i}_ref"]
    _FP7 = _FP7.drop(_FP7.filter(regex='_ref$').columns, axis=1)
    print(f"- size _FP7 with country: {len(_FP7)}, {_FP7.loc[_FP7.stage=='successful', 'funding'].sum()}")
    
    p = _FP7[['generalPic', 'country_code_mapping','country_code']].drop_duplicates()
    print(f"- size de p: {len(p)}")
    #lien part et ref
    p = p.merge(ref, how='outer', on=['generalPic', 'country_code_mapping'], indicator=True).drop_duplicates()
    p = p.loc[p._merge.isin(['both', 'left_only'])]
    # print(f"cols de p: {p.columns}")

    # p1 pic+ccm commun
    p1 = p.loc[p['_merge']=='both'].drop(columns=['_merge', 'country_code'])
    print(f"- size p1 pic+cc: {len(p1)}")

    # p2 pic cc
    p2 = (p.loc[p['_merge']=='left_only'].drop(columns=['_merge', 'id', 'ZONAGE'])
        .merge(ref.rename(columns={'country_code_mapping':'country_code'}), 
                how='inner', on=['generalPic', 'country_code']).drop_duplicates()
        .drop(columns='country_code'))
    print(f"- size p2 pic cc_parent: {len(p2)}")

    # acteurs sans identifiant dont le pic à plusieurs pays ou le pic certaines participations ont un identifiant et pas d'autres 
    p3 = (p.loc[p['_merge']=='left_only'].drop(columns=['_merge', 'country_code_mapping', 'id', 'ZONAGE'])
        .merge(ref, how='inner', on=['generalPic']).drop_duplicates())
    if not p3.empty:
        print(f"1 - A faire si possible, vérifier pourquoi des participations avec pic identiques ont un id ou pas nb pic: {len(p3.generalPic.unique())}")

    if 'p2' in globals() or 'p2' in locals():
        p1 = pd.concat([p1,p2], ignore_index=True).drop_duplicates()
        print(f"2 - size de new p: {len(p)}, cols: {p.columns}") 

    FP7 = (_FP7.drop(columns=['id', 'ZONAGE', 'country_code'])
            .merge(p1[['generalPic', 'country_code_mapping', 'id', 'ZONAGE']], 
                how='left', on=['generalPic', 'country_code_mapping']))
    print(f"- size _FP7 with ref: {len(_FP7)}, size FP7: {len(FP7)},  size with id: {len(FP7.loc[~FP7.id.isnull()])}")
    
    FP7 = FP7.merge(country[['country_code_mapping', 'country_name_mapping', 'country_code']].drop_duplicates(), how='left', on='country_code_mapping')
    # FP7.loc[~FP7.ZONAGE.isnull(), 'country_code'] = FP7.ZONAGE
    if any(FP7.country_code.isnull()):
        print(f"country_code null {FP7.loc[FP7.country_code.isnull(), ['country_code_mapping', 'country_name_mapping']].drop_duplicates()}")
        FP7.loc[FP7.country_code_mapping=='GUF', 'country_code'] = 'FRA'
        FP7.loc[FP7.country_code_mapping=='GUF', 'country_name_mapping'] = 'French Guiana'
        FP7.loc[FP7.country_code_mapping.isin(['SGS', 'IOT']), 'country_code'] = 'GBR'
        FP7.loc[FP7.country_code_mapping=='IOT', 'country_name_mapping'] = 'British Indian Ocean Territory'
        FP7.loc[FP7.country_code_mapping=='SGS', 'country_name_mapping'] = 'South Georgia and the South Sandwich Islands'

    cc = country.drop(columns=['country_code_mapping', 'country_name_mapping', 'countryCode', 'countryCode_parent']).drop_duplicates()
    FP7 = FP7.merge(cc, how='left', on='country_code')
    FP7.loc[FP7.country_code_mapping=='ZOE', 'country_name_mapping'] = 'European organisations area'

    FP7.loc[FP7.country_code_mapping=='ZOE', 'country_code'] = 'ZOE'
    FP7.loc[FP7.country_code=='ZOE', 'country_name_fr'] = 'Union Européenne'
    FP7.loc[FP7.country_code=='ZOE', 'country_name_en'] = 'European organisations area'

    print(f"size FP7 with country assoc: {len(FP7)},\ncols: {FP7.columns}")    
    return FP7
FP7=FP7_cleaning(_FP7, country)

In [ ]:
FP7.loc[(FP7.country_code=='DEU')&(FP7.stage=='successful')&(FP7.pilier!='EURATOM')].funding.sum()
# FP7.pilier.value_counts()

In [ ]:
def FP7_entities(FP7, country):
    print("\n## FP7 entities")
    # part.country_code.unique()
    entities = FP7.loc[~FP7.id.isnull(), ['generalPic','id', 'country_code_mapping']].drop_duplicates()
    print(f"- size entities {len(entities)}")
    if any(entities.id.str.contains(';')):
        entities = entities.assign(id_extend=entities.id.str.split(';')).explode('id_extend')
        entities.loc[(entities.id.str.contains(';', na=False))&(entities.id_extend.str.len()==14), 'id_extend'] = entities.loc[(entities.id.str.contains(';', na=False))&(entities.id_extend.str.len()==14)].id_extend.str[:9]
        entities = entities.drop_duplicates()
        entities_size_to_keep = len(entities)
        print(f"2 - size entities si multi id -> entities_size_to_keep = {entities_size_to_keep}")

    ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
    entities_tmp = merge_ror(entities, ror, country)
    print(f"size entities_tmp after add ror_info: {len(entities_tmp)}, entities_size_to_keep: {entities_size_to_keep}")


    # PAYSAGE
    ### si besoin de charger paysage pickle
    paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
    if any(paysage.groupby('id')['id_clean'].transform('count')>1):
        print(f"1 - paysage doublon oublié: {paysage[paysage.groupby('id')['id_clean'].transform('count')>1][['id', 'id_clean']].sort_values('id')}")
        paysage = paysage.loc[~((paysage.id_clean=='vey7g')&(paysage.id.str.contains('265100057', na=False)))]    
    
    paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
    cat_filter = category_paysage(paysage_category)
    entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)

    sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
    entities_tmp = merge_sirene(entities_tmp, sirene)

    # traitement des id identifiés mais sans referentiels liés
    entities_tmp.loc[(entities_tmp.entities_id.isnull())&(~entities_tmp.id_extend.str.contains('-', na=False)), 'entities_id'] = entities_tmp['id_extend']

    entities_tmp['siren']=entities_tmp.loc[entities_tmp.entities_id.str.contains('^[0-9]{9}$|^[0-9]{14}$', na=False)].entities_id.str[:9]
    entities_tmp.loc[entities_tmp.siren.isnull(), 'siren']=entities_tmp.paysage_siren

    #groupe

    # recuperation tous les siren pour lien avec groupe -> creation var SIREN 
    entities_tmp.loc[~entities_tmp.siren.isnull(), "siren"] = entities_tmp.loc[~entities_tmp.siren.isnull(), "siren"].str.split().apply(set).str.join(";")

    if any(entities_tmp.siren.str.contains(';', na=False)):
        print("1 - ATTENTION faire code pour traiter deux siren différents -> ce qui serait bizarre qu'il y ait 2 siren")
    else:
        ### si besoin de charger groupe
        file_name = f"{PATH_REF}H20_groupe.pkl"
        groupe = pd.read_pickle(file_name)
        print(f"2 - taille de entities_tmp avant groupe:{len(entities_tmp)}")

        entities_tmp=entities_tmp.merge(groupe, how='left', on='siren')

        # entities_tmp.loc[~entities_tmp.groupe_id.isnull(), 'entities_id']= entities_tmp.groupe_id
        # entities_tmp.loc[~entities_tmp.groupe_id.isnull(), 'entities_acronym'] = entities_tmp.groupe_acronym
        # entities_tmp.loc[~entities_tmp.groupe_id.isnull(), 'entities_name'] = entities_tmp.groupe_name

        # entities_tmp.loc[entities_tmp.entities_id.str.contains('gent', na=False), 'siren_cj'] = 'GE_ENT'
        
        # entities_tmp = entities_tmp.drop(['groupe_id','groupe_name','groupe_acronym'], axis=1).drop_duplicates()
        print(f"- size entities_tmp after groupe {len(entities_tmp)}")

    entities_tmp = entities_tmp.merge(get_source_ID(entities_tmp, 'entities_id'), how='left', on='entities_id')
        # traitement catégorie
    # entities_tmp = category_cleaning(entities_tmp, sirene)
    entities_tmp = category_woven(entities_tmp, sirene)
    entities_tmp = category_agreg(entities_tmp)
    return  entities_tmp
entities_tmp=FP7_entities(FP7, country)

In [ ]:
print("\n## FP7 calculation")
print(f"- size part before: {len(FP7)}")
part1 = (FP7[['project_id', 'participant_order', 'role', 'generalPic', 'global_costs',
    'participant_type_code', 'legalName', 'businessName', 'countryCode', 'nutsCode',
    'funding', 'status.x', 'ADRESS', 'city', 'post_code', 'pme', 'stage', 'nom', 'countryCode_parent', 'vat_id',
    'country_code_mapping', 'participant_id', 'number_involved', 'coordination_number', 'id', 'ZONAGE',
    'country_name_mapping', 'country_code', 'country_name_en','country_association_code', 'country_association_name_en',
    'country_group_association_code', 'country_group_association_name_en','country_group_association_name_fr', 
    'country_name_fr', 'article1', 'article2']]
        .merge(entities_tmp, how='left', on=['generalPic', 'country_code_mapping', 'id']))
print(f"- size part before: {len(part1)}")

In [ ]:
part1 = (FP7[['project_id', 'participant_order', 'role', 'generalPic', 'global_costs',
    'participant_type_code', 'legalName', 'businessName', 'countryCode', 'nutsCode',
    'funding', 'status.x', 'ADRESS', 'city', 'post_code', 'pme', 'stage', 'nom', 'countryCode_parent', 'vat_id',
    'country_code_mapping', 'participant_id', 'number_involved', 'coordination_number', 'id', 'ZONAGE',
    'country_name_mapping', 'country_code', 'country_name_en','country_association_code', 'country_association_name_en',
    'country_group_association_code', 'country_group_association_name_en','country_group_association_name_fr', 
    'country_name_fr', 'article1', 'article2']]
        .merge(entities_tmp, how='left', on=['generalPic', 'country_code_mapping', 'id']))

part2=(part1.loc[part1.entities_name.isnull()].drop_duplicates())
part3=(part2.sort_values(['legalName', 'businessName'], ascending=False)
    .groupby(['generalPic', 'country_code_mapping'])
    .first().reset_index()[['generalPic', 'country_code_mapping', 'legalName', 'businessName']]
    .rename(columns={'legalName':'entities_name', 'businessName':'entities_acronym'}))

part2 = (part2.drop(columns=['entities_name', 'entities_acronym', 'nom'])
        .merge(part3, how='left', on=['generalPic', 'country_code_mapping']))
part2['entities_name'] = part2.entities_name.str.capitalize().str.strip()
part2['entities_id'] = "pic"+part2.generalPic.map(str)

part1=part1.loc[~part1.entities_name.isnull()].drop_duplicates()

part1=pd.concat([part1, part2], ignore_index=True).assign(number_involved=1)

part1['nb'] = part1.id.str.split(';').str.len()
for i in ['funding', 'coordination_number', 'number_involved']:
    part1[i] = np.where(part1['nb']>1, part1[i]/part1['nb'], part1[i])

# 'requestedGrant'
print(f"- size part after: {len(part1)}")

if any(part1.entities_id=='nan')|any(part1.entities_id.isnull()):
    print(f"1 - attention il reste des entities sans entities_id valides")

type_entity = pd.read_json(open('data_files/legalEntityType.json', 'r', encoding='UTF-8'))
# part1.loc[part1.participant_type_code=='N/A', 'participant_type_code'] = 'NA'
part1 = (part1.merge(type_entity, how='left', left_on='participant_type_code', right_on='cordis_type_entity_code')
.drop(columns='participant_type_code'))

In [ ]:
part1.loc[(part1.country_code=='DEU')&(part1.stage=='successful')&(part1.project_id.isin(pp))].funding.sum()

In [ ]:
len(part1)

In [ ]:


# # # gestion code nuts
# nuts = pd.read_pickle("data_files/nuts_complet.pkl")
# nuts = (nuts[['nuts_code_2013','nutsCode', 'lvl1Description', 'lvl2Description', 'lvl3Description']]
#         .drop_duplicates()
#         .rename(columns={'nuts_code_2013':'nuts_code_tmp', 'nutsCode':'nuts_code','lvl1Description':'region_1_name', 'lvl2Description': 'region_2_name', 'lvl3Description':'regional_unit_name'}))
# # nuts['region_1_name'] = nuts['region_1_name'].str.title()
# print(len(nuts))

part1['nuts_code_tmp'] = np.where(part1.nutsCode.str.len()<3, np.nan, part1.nutsCode)

print(f"- size part1 with code after cleanup nuts: {len(part1[~part1.nuts_code_tmp.isnull()])}")

nuts = nuts.loc[(nuts.nuts_code_tmp.isin(part1.nuts_code_tmp.unique()))&(~nuts.nuts_code_tmp.isnull())]
part1 = part1.merge(nuts, how='left', on='nuts_code_tmp').drop_duplicates()
print(f"- nuts code without name: {len(part1[(~part1.nuts_code.isnull())&(part1.region_1_name.isnull())])}")

# print(part1.groupby(['stage'], dropna=True )['nuts_code'].size())
print(part1.loc[part1.stage=='successful', 'funding'].sum())


In [ ]:
instr = pd.read_csv('data_files/instru_nomenclature.csv', sep=';')
act=pd.read_json(open("data_files/actions_name.json", 'r', encoding='utf-8'))
msca_correspondence = pd.read_table('data_files/msca_correspondence.csv', sep=";").drop(columns='framework')
erc_correspondence = pd.read_json(open("data_files/ERC_correspondance.json", 'r', encoding='utf-8'))
thema = pd.read_json(open("data_files/thema.json", 'r', encoding='utf-8'))
destination = pd.read_json(open("data_files/destination.json", 'r', encoding='utf-8'))

def themes_cleaning(FP7):
    print("## FP7 themes")
    print(f"- size proj before cleaning: {len(FP7[['project_id', 'stage']].drop_duplicates())}")
    proj = (FP7.assign(stage_name=np.where(FP7.stage=='successful', 'projets lauréats', 'projets évalués'))
            [['project_id', 'stage', 'acronym', 'abstract', 'title', 'call_id', 'stage_name',
            'call_deadline', 'instrument',  'panel_code', 'panel_name', 'call_year', 'duration', 'status_code', 
        'cost_total', 'eu_reqrec_grant', 'free_keywords', 'number_involved', 'submission_date',
        'start_date', 'signature_date', 'end_date',  'pilier', 'prog_abbr', 'prog_lib', 'area_abbr', 'area_lib']]
            .drop_duplicates())

    proj.loc[(proj.prog_abbr=='ERC')&(proj.instrument=='POC'), 'instrument'] = 'ERC-POC'
    proj.loc[proj.prog_abbr=='PEOPLE', 'thema_code'] = 'MSCA'
    proj.loc[proj.prog_abbr=='ERC', 'thema_code'] = 'ERC'

    # print(f"- size proj: {len(proj)}")

    proj = proj.merge(instr, how='left', on='instrument').drop(columns=['name'])
    proj.loc[proj.instrument.str.contains('MC-'), 'action_code'] = 'MSCA'        

    if any(proj.action_code.isnull()):
        print(proj[proj.action_code.isnull()].instrument.unique())   
        
    print(f"- size proj after instru: {len(proj)}")

    # ERC
    proj = proj.merge(erc_correspondence, how='left', left_on=['instrument'], right_on=['old'])

    proj.loc[(proj.thema_code=='ERC')&(proj.destination_code.isnull()), 'destination_code'] = 'ERC-OTHER'

    proj.loc[proj.thema_code=='ERC', 'programme_code'] = 'ERC'
    proj.loc[proj.thema_code=='ERC', 'programme_name_en'] = 'European Research Council (ERC)'
    print(f"- size proj after ERC: {len(proj)}")

    # MSCA
    proj = proj.merge(msca_correspondence, how='left', left_on=['instrument'], right_on=['old'])
    proj.loc[proj.call_id.str.contains('NIGHT'), 'destination_detail_code'] = 'CITIZENS'
    proj.loc[~proj.destination_detail_code.isnull(), 'destination_code'] = proj.destination_detail_code.str.split('-').str[0]
    proj.loc[(proj.destination_code.isnull())&(proj.thema_code=='MSCA'), 'destination_code'] = 'MSCA-OTHER'
    proj.loc[proj.thema_code=='MSCA', 'programme_code'] = 'MSCA'
    proj.loc[proj.thema_code=='MSCA', 'programme_name_en'] = 'Marie Skłodowska-Curie Actions (MSCA)'

    proj.rename(columns={'instrument':'fp_specific_instrument'}, inplace=True)

    print(f"- size proj success after msca: {proj.loc[proj.stage=='successful'].project_id.nunique()}, nb project_id: {len(proj.loc[proj.stage=='successful'])}")
    print(f"- size proj after msca: {len(proj)}")
    #euratom
    proj.loc[(proj.pilier.isin(['EURATOM']))&(proj.prog_abbr=='Fission'), 'programme_code'] = 'NFRP'
    proj.loc[(proj.pilier.isin(['EURATOM']))&(proj.programme_code=='NFRP'), 'programme_name_en'] = 'Nuclear fission and radiation protection'
    proj.loc[proj.prog_abbr=='Fusion', 'programme_code'] = 'Fusion'
    proj.loc[proj.prog_abbr=='Fusion', 'programme_name_en'] = 'Fusion Energy'

    euratom = pd.read_csv('data_files/euratom_thema_all_FP.csv', sep=';', na_values='')
    proj = proj.merge(euratom[['topic_area', 'thema_code', 'thema_name_en']], how='left', left_on='area_abbr', right_on='topic_area', suffixes=['', '_t'])
    proj.loc[(~proj.thema_code_t.isnull()), 'thema_code'] = proj.loc[(~proj.thema_code_t.isnull()), 'thema_code_t']
    proj = proj.filter(regex=r'.*(?<!_t)$')
    print(f"- size proj after euratom: {len(proj)}")

    #ju_jti
    proj.loc[proj.prog_abbr=='SP1-JTI', 'thema_code'] = 'JU-JTI'
    proj.loc[proj.prog_abbr=='SP1-JTI', 'destination_code'] = proj.area_abbr.str.split('-').str[-1]
    proj.loc[proj.area_abbr=='JTI-CS', 'destination_code'] = 'CLEAN-AVIATION'

    proj.loc[(proj.destination_code=='CLEAN-SKY'), 'destination_code'] = 'CLEAN-AVIATION'
    proj.loc[(proj.destination_code=='FCH'), 'destination_code'] = 'CLEANH2'
    proj.loc[(proj.destination_code=='IMI'), 'destination_code'] = 'IHI'
    proj.loc[(proj.destination_code.isin(['ENIAC','ARTEMIS'])), 'destination_code'] = 'Chips'
    proj.loc[proj.thema_code=='JU-JTI', 'action_code'] = proj.fp_specific_instrument.str.split('-').str[1]
    print(f"- size proj after ju-jti: {len(proj)}")

    # WIDENING COST
    proj.loc[proj.area_abbr.str.contains('COST', na=False), 'thema_code'] = 'COST'
    proj.loc[proj.area_abbr.str.contains('COST', na=False), 'programme_code'] = 'Widening'
    proj.loc[proj.area_abbr.str.contains('COST', na=False), 'programme_name_en'] = 'Widening participation and spreading excellence'
    print(f"- size proj after cost: {len(proj)}")

    proj.loc[proj.pilier=='EURATOM', 'pilier_name_en'] = 'Euratom'
    proj.loc[(proj.prog_abbr.isin(['PEOPLE','ERC']))|(proj.prog_abbr=='INFRA'), 'pilier_name_en'] = 'Excellent Science'
    proj.loc[proj.pilier_name_en.isnull(), 'pilier_name_en'] = proj.pilier.str.capitalize()
    print(f"- size proj after cost: {len(proj)}")

    proj.loc[proj.programme_code.isnull(), 'programme_code'] = proj.prog_abbr
    proj.loc[proj.programme_name_en.isnull(), 'programme_name_en'] = proj.prog_lib
    print(f"- size proj after cost: {len(proj)}")

    proj.loc[(~proj.thema_code.isin(['MSCA','ERC']))&(proj.destination_code.isnull()), 'destination_code'] = proj.area_abbr
    proj.loc[proj.destination_code.isnull(), 'destination_code'] = proj.thema_code+'-OTHER'
    proj = proj.merge(destination[['destination_code', 'destination_name_en']], how='left', on='destination_code')
    proj = (proj
            .merge(destination.rename(columns={'destination_code':'destination_detail_code', 'destination_name_en':'destination_detail_name_en'})
            [['destination_detail_code', 'destination_detail_name_en']], how='left', on='destination_detail_code')
            .drop_duplicates())
    print(f"- size proj after cost: {len(proj)}")

    proj.loc[(~proj.thema_code.isin(['MSCA','ERC']))&(proj.destination_name_en.isnull()), 'destination_name_en'] = proj.area_lib
    proj.loc[proj.thema_code.isnull(), 'thema_code'] = proj.prog_abbr
    proj = proj.merge(thema[['thema_code', 'thema_name_en']], how='left', on='thema_code', suffixes=['', '_t'])
    proj.loc[proj.thema_name_en.isnull(), 'thema_name_en'] = proj.thema_name_en_t
    proj.loc[proj.thema_name_en.isnull(),'thema_name_en'] = proj.prog_lib
    proj = proj.filter(regex=r'.*(?<!_t)$')
    print(f"- size proj after thema_code: {len(proj)}")

    proj = (proj.drop(columns=['area_abbr', 'area_lib'])
            .rename(columns={'prog_lib':'fp_specific_programme', 'pilier':'fp_specific_pilier'}))
    
    print(proj[['programme_code',
    'programme_name_en', 'thema_name_en', 'destination_code', 'destination_name_en',
    'destination_detail_code','destination_detail_name_en']].drop_duplicates())
    print(len(proj))
    return proj
proj=themes_cleaning(FP7)

In [ ]:
def proj_cleaning(proj):
    print(f"- size proj before cleaning: {len(proj)}")
    proj = proj.merge(act, how='left', on='action_code')
    proj = proj.merge(call, how='left', on='call_id').assign(ecorda_date=pd.to_datetime('2021-04-30'), framework='FP7')
    proj = proj.assign(ecorda_date=pd.to_datetime('2021-04-30'), framework='FP7')
    for i in ['title', 'abstract', 'free_keywords']:
        proj[i]=proj[i].str.replace('\\n|\\t|\\r|\\s+', ' ', regex=True).str.strip()

    kw = proj[['project_id', 'free_keywords']]
    kw = kw.assign(free_keywords = kw.free_keywords.str.split(';|,')).explode('free_keywords')
    kw = kw.loc[kw.free_keywords.str.len()>3].drop_duplicates()
    kw.free_keywords = kw.free_keywords.groupby(level=0).apply(lambda x: '|'.join(x.str.strip().unique()))

    proj = proj.drop(columns='free_keywords').merge(kw.drop_duplicates(), how='left', on='project_id')
    proj.mask(proj=='', inplace=True)  

    for d in ['call_deadline', 'signature_date',  'start_date',  'end_date', 'submission_date']:
        proj[d] = pd.to_datetime(proj[d],format='%d/%m/%Y %H:%M:%S')
    print(f"- size proj cleaned: {len(proj)}")
    return proj
proj=proj_cleaning(proj)
# def proj_ods(proj, part1):
#     country=(part1.loc[part1.stage=='successful',
#                 ['project_id','country_code','country_name_fr','country_code_mapping', 'ZONAGE',
#                     'country_name_mapping', 'nuts_code', 'region_1_name', 'region_2_name','regional_unit_name']]
#         .drop_duplicates()
#         .groupby(['project_id'], as_index = False).agg(lambda x: ';'.join(map(str,filter(None, x))))
#         .drop_duplicates())

#     prop = (proj.loc[proj.stage=='evaluated', ['project_id', 'cost_total', 'eu_reqrec_grant', 'number_involved']]
#         .rename(columns={'number_involved':'proposal_numberofapplicants', 'eu_reqrec_grant':'proposal_requestedgrant', 'cost_total':'proposal_budget'})
#         .drop_duplicates())

#     p = (proj.loc[proj.stage=='successful', ['project_id', 'eu_reqrec_grant', 'number_involved', 'cost_total']]
#         .rename(columns={'eu_reqrec_grant':'project_eucontribution', 'number_involved':'project_numberofparticipants','cost_total':'project_totalcost'})
#         .drop_duplicates())

#     # # PROVISOIRE quand def call refonctionnera
#     # proj=proj.assign(call_budget=np.nan)

#     project = (proj.loc[proj.stage=='successful', 
#             ['abstract', 'acronym', 'action_code', 'action_name', 'call_budget','call_deadline', 'call_id', 'call_year',
#             'destination_code','destination_detail_code', 'destination_detail_name_en', 'destination_name_en', 
#             'duration', 'ecorda_date', 'end_date', 'fp_specific_instrument', 'framework', 'free_keywords', 
#             'panel_code', 'panel_name', 'fp_specific_programme', 'fp_specific_pilier',
#             'pilier_name_en', 'programme_code', 'programme_name_en', 'project_id', 'signature_date', 'stage', 'stage_name', 
#             'start_date', 'status_code', 'submission_date', 'thema_code', 'thema_name_en', 'title']]
            
#         .drop_duplicates())

#     project = project.merge(p, how='left', on='project_id').merge(country, how='inner', on='project_id').merge(prop, how='left' , on='project_id')

#     print(f"1 - size project lauréats: {len(project)}, {len(p)}, fund: {'{:,.1f}'.format(p['project_eucontribution'].sum())}")

#     with open(f"{PATH_CLEAN}FP7_successful_projects.pkl", 'wb') as file:
#         pd.to_pickle(project, file)
#     return project
# proj_ods(proj, part1)

def FP7_all(proj, part1):
    t = (proj.drop(columns=['cost_total', 'duration', 'end_date', 'eu_reqrec_grant', 'fp_specific_instrument', 
                        'fp_specific_programme', 'fp_specific_pilier',
                        'number_involved', 'signature_date', 'start_date', 'submission_date'])
        .merge(part1, how='inner', on=['project_id', 'stage'])
        .rename(columns={'funding':'calculated_fund', 'ZONAGE':'extra_joint_organization'}))
    
    t = (t.assign(is_ejo=np.where(t.extra_joint_organization.isnull(), 'Sans', 'Avec')))

    t.loc[(t.destination_code.isin(['PF', 'ERARESORG', 'GA']))|((t.thema_code.isin(['ERC', 'COST']))&(t.destination_code!='SyG')), 'coordination_number'] = 0
    t=t.assign(with_coord=True)
    t.loc[(t.destination_code.isin(['PF', 'ERARESORG', 'GA']))|((t.thema_code.isin(['ERC', 'COST']))&(t.destination_code!='SyG')), 'with_coord'] = False

    t.loc[t.thema_code=='ERC', 'erc_role'] = 'partner'

    t.loc[(t.destination_code=='SyG'), 'erc_role'] = 'PI'
    t.loc[(t.action_code=='ERC')&(t.destination_code!='SyG')&(t.role=='coordinator'), 'erc_role'] = 'PI'
    t.loc[(t.destination_code=='ERC-OTHER'), 'erc_role'] = np.nan


    file_name = f"{PATH_CLEAN}FP7_data.pkl"
    with open(file_name, 'wb') as file:
        pd.to_pickle(t, file)

    print(f"size proj: {t.loc[t.stage=='successful'].project_id.nunique()}, nb project_id: {len(t.loc[t.stage=='successful'])}, {t.loc[t.stage=='successful', 'calculated_fund'].sum()}")
    return t
t=FP7_all(proj, part1)

In [ ]:
t.loc[(t.country_code=='DEU')&(t.stage=='successful')&(t.call_year=='2007')&(t.pilier_name_en!='Euratom')].calculated_fund.sum()



In [ ]:
from functions_shared import *
t=t.drop_duplicates().loc[(t.stage=='successful')&(t.pilier_name_en!='Euratom')]
print(len(t))
x=pd.crosstab(t['country_code'], t['call_year'], values=t['calculated_fund'], aggfunc='sum',margins=True, margins_name= 'All').reset_index()
work_csv(x, 'fp7_count')